# Lab3b Magnetometer Data Analysis
Contents of this notebook are developed by Dogacan Su Ozturk for the GEOS 419 Lecture. 

# This notebook

1. Importing essential Python libraries
2. Loading EZIE Magnetometer Data
   - Importing helper functions
   - Unzipping data files
   - Copying data to a folder
   - Reading the text files into a Pandas dataframe
3. Downloading GIMA and USGS Magnetometer Data
   - Importing PySpedas
   - Downloading data for Poker Flat (GIMA) and College (USGS) sites
4. Comparing EZIE measurements with the science grade magnetometers
   - Plotting data with Matplotlib
   - Removing baseline
   - Labelling and saving figures


### 1. Importing essential Python libraries
Most of the functionality of this notebook comes from importing essential Python libraries. I am now assuming you are quite comfortable with the Python libraries, therefore I will only provide a brief summary of their functionality. 

In [ ]:
# Essential Python libraries for this exercise
import numpy as np
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
myFmt = mdates.DateFormatter('%D/%m %H:%M')
#%matplotlib widget 

### 2. Loading EZIE Magnetometer Data
In order to reduce cluttering, I have included the functionality that you would need to read and parse EZIE magnetometer data files in a separate file called 'ezie_mag_helper_functions'. You can import different functions from this helper library to unzip and merge different data files. To learn more about functionality you can use the standard Python syntax. These libraries are imported as follows.

In [ ]:
# Libraries for unzipping eziemag data
from ezie_mag_helper_functions import unzip_ezie_files

First we will start with the unzip_ezie_files function. Once you download the data using the eziemag.local interface, you will notice the files are zipped in a specific format. The functions you will use are geared towards 60 second averaged files. You will need to develop your own helper functions or modify the existing functionality by changing the folderpath if you wish to use 1 second data. In the examples below, we are using the unzip_ezie_files function. This function takes three parameters, the first one being the keyword string assigned to the magnetometer, the second is a string of folderpath, and the third is a logical flag if you want to copy all individual daily measurements unzipped into one big folder. By design, this folder will be placed in the folderpath, with a name appending the keyword with '_merged'.

In [ ]:
geos1_filepath = unzip_ezie_files('geos1','GEOS_419_1/Deployment_00_20241216_20/', merge=True)
geos2_filepath = unzip_ezie_files('geos2','GEOS_419_2/Deployment_00_20241216_20/', merge=True)
geos3_filepath = unzip_ezie_files('geos3','GEOS_419_3/Deployment_00_20241216_20/', merge=True)

The code cell when executed will return print statements to help users locate filepath errors. Now that a new folder for unzipped files are generated, we can start reading the data. We will use the merge_eziemag_data function to read all the measurement data obtained. This function only needs the folderpath, which the unzip_eziemag_data provides. This function, calls another function named read_and_parse_ezie_mag_data. If user wants to read and parse single day data, they can pass the folderpath for this folder instead. 

In [ ]:
# Libraries for merging eziemag data
from ezie_mag_helper_functions import merge_eziemag_data

We can now pass the path to function to generate a Pandas Dataframe for measurements from each different EZIE MAG kit. 

In [ ]:
geos1_data = merge_eziemag_data(geos1_filepath)
geos2_data = merge_eziemag_data(geos2_filepath)
geos3_data = merge_eziemag_data(geos3_filepath)

Now the measurement data is parsed and read into Pandas Dataframes. You can use generic Pandas functionality on these variables. I will explore some of these functionality below.

In [ ]:
geos1_data.describe()

In [ ]:
geos1_data.head()

In [ ]:
geos1_data.columns

In [ ]:
geos1_data['Bx'].plot()

### 3. Downloading GIMA and USGS Magnetometer Data
Next step to investigate our EZIE magnetometer measurements is to understand their accuracy. For this, we need to download magnetometer measurements for scientific grade instruments. There are two magnetometer stations one from Geophysical Institute Magnetometer Array (GIMA) at Poker Flat, AK and another from U.S. Geological Survey at College, AK. The widely used code names for these magnetometers are 'PKR'/'POKR' and 'CMO' respectively. We will use the SPEDAS package to download the data for these two sites.

In [ ]:
# Libraries for downloading professional grade magnetometer measurements
import pyspedas 

Then we will need to set a time range, using start and stop times. These times are passed as strings to the pyspedas.themis.gmag library. You can instead use any start_time and end_time in datetime.datetime format. Here, I am using the first and last time entries in the geos1_data to match the data with our measurements.

In [ ]:
#set start and end time for data analysis
start_time = pd.to_datetime(geos1_data.timeString.iloc[0])
end_time = pd.to_datetime(geos1_data.timeString.iloc[-1])
trange = [start_time.strftime('%Y-%m-%d/%H:%M:%S'), end_time.strftime('%Y-%m-%d/%H:%M:%S')]

Let's first download the data from Poker Flat station. We can pass sites=['pokr', 'cmo'] and download data simultaneously but for demonstraton purposes, I will download and assign data separately.

In [ ]:
poker_data = pyspedas.themis.gmag(trange=trange, sites=['pokr'], 
                                level='l2', time_clip=True,notplot=True)


Let's check what the data looks like.

In [ ]:
poker_data

The data is read into a dictionary. This is different than the Pandas Dataframe. Here the 'x' is unixtime, 'y' is the magnetometer measurements, with each column representing Bx, By, Bz. Now, we will do the same for College site.

In [ ]:
college_data = pyspedas.themis.gmag(trange=trange, sites=['cmo'], 
                                level='l2', time_clip=True,notplot=True)

### 4. Comparing EZIE measurements with the science grade magnetometers
Now that we have our own EZIE magnetometer measurements and science grade measurements, let's plot and compare the results. Below I will provide lines of code required to plot different components of the data set. Feel free to modify them as you see fit for your purposes. 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,5))

ax.plot(geos1_data['timeString'], geos1_data['Bx'], color='#FF9FE5', label='GEOS-1')
ax.plot(geos2_data['timeString'], geos2_data['Bx'], color='#6E0B34', label='GEOS-2')
ax.plot(geos3_data['timeString'], geos3_data['Bx'], color='#FF858D', label='GEOS-3')

ax.set_ylabel('Magnetic Field [nT]', fontsize=14)
ax.set_xlabel('Universal Time [D/M HH:MM]', fontsize=14)
ax.set_title('Magnetic Field Measurements - X Component', fontsize=16)

# Set the date format on the x-axis
date_format = mdates.DateFormatter('%d/%m %H:%M')
ax.xaxis.set_major_formatter(date_format)

# Optionally, you can set the locator for the ticks
ax.xaxis.set_major_locator(mdates.HourLocator(interval=12))

ax.legend(loc=4, fontsize=12)
ax.grid()


Before we save the plot, there are a few things that require our attention. The first one is the initial perturbation, or a period called warm-up, where sensors are still responding to perturbations after being plugged in and being contaminated with us walking around, setting them up. This data is not useable and needs to be removed. The second one is the offset we have between each magnetometer. These magnetometer kits are sometimes referred to as 'variometers' because they don't provide absolute measurements but variations. Therefore, it is common practice to remove either the first value in the measurements or the median of the duration. Since our first value is in the contaminated interval, we can remove those values first, and then remove the median as the baseline.  

In [ ]:
geos1_data.Bx.head(10), geos2_data.Bx.head(10), geos3_data.Bx.head(10)

We can see what we regard as "contaminated period" has different durations for each magnetometer kit. We can either remove the first 10 minutes from all measurements or remove values based on variability of this duration.

In [ ]:
geos1_data.drop(geos1_data.index[:10], inplace=True)
geos2_data.drop(geos2_data.index[:10], inplace=True)
geos3_data.drop(geos3_data.index[:10], inplace=True)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,5))

ax.plot(geos1_data['timeString'], geos1_data['Bx']-geos1_data['Bx'].median(), color='#FF9FE5', label='GEOS-1')
ax.plot(geos2_data['timeString'], geos2_data['Bx']-geos2_data['Bx'].median(), color='#6E0B34', label='GEOS-2')
ax.plot(geos3_data['timeString'], geos3_data['Bx']-geos3_data['Bx'].median(), color='#FF858D', label='GEOS-3')

ax.set_ylabel('Magnetic Field [nT]', fontsize=14)
ax.set_xlabel('Universal Time [D/M HH:MM]', fontsize=14)
ax.set_title('Magnetic Field Measurements - X Component', fontsize=16)

ax.set_ylim([-500,1500])
# Set the date format on the x-axis
date_format = mdates.DateFormatter('%d/%m %H:%M')
ax.xaxis.set_major_formatter(date_format)

# Optionally, you can set the locator for the ticks
ax.xaxis.set_major_locator(mdates.HourLocator(interval=12))

ax.legend(loc=1, fontsize=12)
ax.grid()

Now let's add the Poker Flat and College site data to this plot. First we need to convert the UnixTime to UT time.

In [ ]:
poker_time_array = [dt.datetime(1970, 1, 1) + dt.timedelta(seconds=seconds) for seconds in poker_data['thg_mag_pokr']['x']]
college_time_array = [dt.datetime(1970, 1, 1) + dt.timedelta(seconds=seconds) for seconds in college_data['thg_mag_cmo']['x']]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,5))

ax.plot(geos1_data['timeString'], geos1_data['Bx']-geos1_data['Bx'].median(), color='#FF9FE5', label='GEOS-1')
ax.plot(geos2_data['timeString'], geos2_data['Bx']-geos2_data['Bx'].median(), color='#6E0B34', label='GEOS-2')
ax.plot(geos3_data['timeString'], geos3_data['Bx']-geos3_data['Bx'].median(), color='#FF858D', label='GEOS-3')

ax.plot(poker_time_array, poker_data['thg_mag_pokr']['y'][:,0]-np.median(poker_data['thg_mag_pokr']['y'][:,0]), color='#2B50AA', label='POKR')
ax.plot(college_time_array, college_data['thg_mag_cmo']['y'][:,0]-np.median(college_data['thg_mag_cmo']['y'][:,0]), color='#77C77C', label='CMO')

ax.set_ylabel('Magnetic Field [nT]', fontsize=14)
ax.set_xlabel('Universal Time [D/M HH:MM]', fontsize=14)
ax.set_title('Comparison of the Magnetic Field Measurements - X Component', fontsize=16)

ax.set_ylim([-500,1500])
# Set the date format on the x-axis
date_format = mdates.DateFormatter('%d/%m %H:%M')
ax.xaxis.set_major_formatter(date_format)

# Optionally, you can set the locator for the ticks
ax.xaxis.set_major_locator(mdates.HourLocator(interval=12))

ax.legend(loc=1, fontsize=12)
ax.grid()

Before we start analyzing the comparison in more depth there are a few more considerations we need to take into account. The first one is the overlapping of lines. We can introduce the alpha, transparency parameter, to show overlapping lines  better. 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,5))

ax.plot(geos1_data['timeString'], geos1_data['Bx']-geos1_data['Bx'].median(), color='#FF9FE5', alpha=0.6, label='GEOS-1')
ax.plot(geos2_data['timeString'], geos2_data['Bx']-geos2_data['Bx'].median(), color='#6E0B34', alpha=0.6, label='GEOS-2')
ax.plot(geos3_data['timeString'], geos3_data['Bx']-geos3_data['Bx'].median(), color='#FF858D', alpha=0.6, label='GEOS-3')

ax.plot(poker_time_array, poker_data['thg_mag_pokr']['y'][:,0]-np.median(poker_data['thg_mag_pokr']['y'][:,0]), color='#2B50AA', alpha=0.6, label='POKR')
ax.plot(college_time_array, college_data['thg_mag_cmo']['y'][:,0]-np.median(college_data['thg_mag_cmo']['y'][:,0]), color='#77C77C', alpha=0.6, label='CMO')

ax.set_ylabel('Magnetic Field [nT]', fontsize=14)
ax.set_xlabel('Universal Time [D/M HH:MM]', fontsize=14)
ax.set_title('Comparison of the Magnetic Field Measurements - X Component', fontsize=16)

ax.set_ylim([-500,1500])
# Set the date format on the x-axis
date_format = mdates.DateFormatter('%d/%m %H:%M')
ax.xaxis.set_major_formatter(date_format)

# Optionally, you can set the locator for the ticks
ax.xaxis.set_major_locator(mdates.HourLocator(interval=12))

ax.legend(loc=1, fontsize=12)
ax.grid()

What adding the alpha parameter showed us is that:
- The warm-up period in EZIE magnetometer kits is longer than 10 minutes. However, the peak we see around 23.40 UT is seen in the CMO measurements too, which indicates it is likely to be a real variation. 
- Poker Flat data is missing and only starts around 2024-12-18, 19:10 UT. 
- The sharp peak toward the end of the interval is seen by all three EZIE magnetometer kits. It could potentially be an artificial signature and needs to be further investigated before being removed as an outlier.

Now, we can save the figure. 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,5))

ax.plot(geos1_data['timeString'], geos1_data['Bx']-geos1_data['Bx'].median(), color='#FF9FE5', alpha=0.6, label='GEOS-1')
ax.plot(geos2_data['timeString'], geos2_data['Bx']-geos2_data['Bx'].median(), color='#6E0B34', alpha=0.6, label='GEOS-2')
ax.plot(geos3_data['timeString'], geos3_data['Bx']-geos3_data['Bx'].median(), color='#FF858D', alpha=0.6, label='GEOS-3')

ax.plot(poker_time_array, poker_data['thg_mag_pokr']['y'][:,0]-np.median(poker_data['thg_mag_pokr']['y'][:,0]), color='#2B50AA', alpha=0.6, label='POKR')
ax.plot(college_time_array, college_data['thg_mag_cmo']['y'][:,0]-np.median(college_data['thg_mag_cmo']['y'][:,0]), color='#77C77C', alpha=0.6, label='CMO')

ax.set_ylabel('Magnetic Field [nT]', fontsize=14)
ax.set_xlabel('Universal Time [D/M HH:MM]', fontsize=14)
ax.set_title('Comparison of the Magnetic Field Measurements - X Component', fontsize=16)

ax.set_ylim([-500,1500])
# Set the date format on the x-axis
date_format = mdates.DateFormatter('%d/%m %H:%M')
ax.xaxis.set_major_formatter(date_format)

# Optionally, you can set the locator for the ticks
ax.xaxis.set_major_locator(mdates.HourLocator(interval=12))

ax.legend(loc=1, fontsize=12)
ax.grid()

fig.savefig('./magnetometer_data_comparison_{0:%Y_%m_%d_%H}.png'.format(geos1_data['timeString'].iloc[0]), bbox_inches='tight', dpi=200)